In [ ]:
!pip install grpcio grpcio-tools


In [ ]:
%%writefile my_service.proto
syntax = "proto3";

package my_service;

service MyService {
  rpc Command (CommandRequest) returns (CommandRequest) {}
}

message CommandRequest {
  string command = 1;
}

message CommandResponse {
  string command_output = 1;
}


In [ ]:
!python -m grpc_tools.protoc -I./ --python_out=. --grpc_python_out=. my_service.proto


In [ ]:
!ls
!python my_service_pb2_grpc.py

In [ ]:
import grpc
import my_service_pb2
import my_service_pb2_grpc
from  concurrent import futures

import subprocess

class MyService(my_service_pb2_grpc.MyServiceServicer):
    def Command(self, request, context):
        result = subprocess.run(request.command.split(), stderr=subprocess.PIPE, text=True)
        return my_service_pb2.CommandResponse(message=f"command output: {result}")

def serve():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    my_service_pb2_grpc.add_MyServiceServicer_to_server(MyService(), server)
    server.add_insecure_port("[::]:50051")
    server.start()
    server.wait_for_termination()

if __name__ == "__main__":
    serve()


In [ ]:
import grpc
import my_service_pb2
import my_service_pb2_grpc

def run():
    with grpc.insecure_channel("localhost:50051") as channel:
      while True:
        msg = input("Command ('quit' to exit): ")
        if msg == 'quit':
          break
        else:
          stub = my_service_pb2_grpc.MyServiceStub(channel)
          response = stub.Command(my_service_pb2.CommandRequest(command=msg))
          print(response.message)

if __name__ == "__main__":
    run()
